### Rating a song in the msd database (obsolete)
This was a first attempt to rate the offensiveness of a song in the database.
To do this, we need to
- identify an individual song
- get all lyrics corresponding to this song
- rate them by offensiveness

In this notebook, we are investigating the structure of the data.
But our approach is not fast enough.

In [1]:
import sqlite3

In [3]:
conn = sqlite3.connect("../datasets/mxm_dataset.db")

The database contains the following:

https://github.com/tbertinmahieux/MSongsDB/blob/master/Tasks_Demos/Lyrics/README.txt
_More details on the database:
   - it contains two tables, 'words' and 'lyrics'
   - table 'words' has one column: 'word'. Words are entered according
     to popularity, check their ROWID if you want to check their position.
     ROWID is an implicit column in SQLite, it starts at 1.
   - table 'lyrics' contains 5 columns, see below
   - column 'track_id' -> as usual, track id from the MSD
   - column 'mxm_tid' -> track ID from musiXmatch
   - column 'word' -> a word that is also in the 'words' table
   - column 'cnt' -> word count for the word
   - column 'is_test' -> 0 if this example is from the train set, 1 if test_
   
We want to connect our insights to the million song database and its metadata.
Therefore we want to use the track_id to identify songs.

The lyrics table contains individual entries for every 

In [4]:
# this is the content of the lyrics table
# please note that it's not "cnt" but "count", the README is wrong
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(lyrics);")
print(cursor.fetchall())
cursor.close()

[(0, 'track_id', '', 0, None, 0), (1, 'mxm_tid', 'INT', 0, None, 0), (2, 'word', 'TEXT', 0, None, 0), (3, 'count', 'INT', 0, None, 0), (4, 'is_test', 'INT', 0, None, 0)]


### Use SQL to extract information

We want to have all track ids, and for every track id, we need the words and counts

The data is big, but even as a pandas DataFrame, the size stays below 5GB.
I think the comfort of pandas is enough to warrant loading this into memory.

In [5]:
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT track_id FROM lyrics ORDER BY track_id;")
track_ids = cursor.fetchall()
cursor.close()
track_ids[:5]

[('TRAAAAV128F421A322',),
 ('TRAAABD128F429CF47',),
 ('TRAAAED128E0783FAB',),
 ('TRAAAEF128F4273421',),
 ('TRAAAEW128F42930C0',)]

In [6]:
cursor = conn.cursor()
cursor.execute("SELECT track_id, word, count FROM lyrics ORDER BY track_id;")
track_word_count = cursor.fetchall()
cursor.close()
track_word_count[:5]

[('TRAAAAV128F421A322', 'i', 6),
 ('TRAAAAV128F421A322', 'the', 4),
 ('TRAAAAV128F421A322', 'you', 2),
 ('TRAAAAV128F421A322', 'to', 2),
 ('TRAAAAV128F421A322', 'and', 5)]

In [7]:
conn.close()

In [8]:
import pandas as pd

In [9]:
id_series = pd.Series(track_ids)
track_ids[:5]

[('TRAAAAV128F421A322',),
 ('TRAAABD128F429CF47',),
 ('TRAAAED128E0783FAB',),
 ('TRAAAEF128F4273421',),
 ('TRAAAEW128F42930C0',)]

In [10]:
sqldb_frame = pd.DataFrame(track_word_count, columns=["track_id", "word", "count"])
del track_word_count

### Constructing a table to hold song ratings

We want to create a table which allows intuitive indexing into the rating of a song.

The table will contain the frequency of each word category. We set up a multiindex to allow slicing along the different characteristics of the word

In [11]:
word_table = pd.read_pickle("../pickles/word_table_cleaned.pickle")

In [12]:
word_table.head()

,category,strength,target
word,,,
bonk,non-discriminatory,mild,None
bukkake,non-discriminatory,strong,None
cocksucker,non-discriminatory,strong,None
dildo,non-discriminatory,strong,None
ho,non-discriminatory,strong,None


In [13]:
index_tuples=[]

for strength in ["mild", "medium", "strong", "strongest"]:
    for category in ["discriminatory", "non-discriminatory"]:
        for target in ["None", "race", "mental or physical ability", "sexuality"]:
            index_tuples.append([strength, category, target])

In [14]:
index = pd.MultiIndex.from_tuples(index_tuples, names=["strength", "category", "target"])

In [15]:
rating_frame = pd.DataFrame(index=["track_id"], columns = index)

In [16]:
rating_frame

strength           mild                                            \
category discriminatory                                             
target             None race mental or physical ability sexuality   
track_id            NaN  NaN                        NaN       NaN   

strength                                                               \
category non-discriminatory                                             
target                 None race mental or physical ability sexuality   
track_id                NaN  NaN                        NaN       NaN   

strength         medium         ...                        strong            \
category discriminatory         ...            non-discriminatory             
target             None race    ...    mental or physical ability sexuality   
track_id            NaN  NaN    ...                           NaN       NaN   

strength      strongest                                            \
category discriminatory                                             
target             None race mental or physical ability sexuality   
track_id            NaN  NaN                        NaN       NaN   

strength                                                               
category non-discriminatory                                            
target                 None race mental or physical ability sexuality  
track_id                NaN  NaN                        NaN       NaN  

[1 rows x 32 columns]

## Populate the rating frame

### First let's look at a sample song

In [17]:
for track_id in id_series:
    print(track_id)
    break

('TRAAAAV128F421A322',)


In [18]:
mentioned_words = sqldb_frame[sqldb_frame["track_id"]==track_id[0]]

In [19]:
def fill_entry(mentioned_words):

    entry = {}
    total_count = 0

    for index, row in mentioned_words.iterrows():
        word = row["word"]
        count = row["count"]
        total_count+=count

        if word in word_table.index:
            rating = word_table.loc[word]
            strength = rating["strength"]
            category = rating["category"]
            target = rating["target"]

            if target==None:
                target="None"
            
            index = (strength, category, target)
            if index in entry:
                entry[index] += count
            else:
                entry[index] = count
                
    if total_count >0:
        normalized={}
        for key, value in entry.items():
            normalized[key] = value / total_count


        return normalized
    else:
        return entry

fill_entry(mentioned_words)

{('mild', 'non-discriminatory', 'None'): 0.009708737864077669}

In [20]:
5

5

### Now add all songs to the rating_frame
This takes much too long.

The hotspot is searching for the words in this track.
Maybe we can have a group by or something similar before this ?

Or we need to run this on the cluster.

In [21]:
from tqdm import tqdm

In [22]:
for track_id in tqdm(id_series):
    track_id = track_id[0]
    mentioned_words = sqldb_frame[sqldb_frame["track_id"]==track_id]
    entry = fill_entry(mentioned_words)
    
    if entry == None:
        print("bad: "+track_id)
    else:
        rating_frame.loc[track_id, :]=entry

  0%|          | 45/237662 [00:48<72:03:02,  1.09s/it]

KeyboardInterrupt: 